# Build a "no internet" TRex Game

![](images/dino-game.png "Dino")

# Define the game

A 2D, Side-scrolling, infinite, 1 key control video game. It has two scenes, game and game end.

# Game Loop

The game may have multiple round. At the end of game, player can press 'ESC' to quit the game or 'Enter' to do another game.

In [ ]:
if __name__ == '__main__':
    # setup the screen ....
    
    while(True):
        main_game = TRexGame(window)
        score = main_game.start()
        if not should_continue(window, score):
            break

In [ ]:
def should_continue(window, score):
    window.clear()
    window.border(NO_BORDER)
    window.addstr(10, 27, "  ___   _   __  __ ___    _____   _____ ___ ")
    window.addstr(11, 27, " / __| /_\ |  \/  | __|  / _ \ \ / / __| _ \\")
    window.addstr(12, 27, "| (_ |/ _ \| |\/| | _|  | (_) \ V /| _||   /")
    window.addstr(13, 27, " \___/_/ \_\_|  |_|___|  \___/ \_/ |___|_|_\\")
    window.addstr(15, 27, "             FINAL_SCORE : "+str(score))
    window.addstr(17, 27, "Press 'Enter' Key to Restart or 'ESC' to Quit")
    while(True):
        key_event = window.getch()
        if key_event is KEY_ESC:
            return False
        elif key_event is KEY_ENTER:
            return True

## Inner Loop

# Graphic

## Sprites / Actors / Game Objects

![](images/sprites.png)

## Animation

## Background

### Parallax scrolling

![](images/parallax_scroll.gif "Parallel Scrolling")

# Control

# Scoring

# Collision detection 

# Game Plan

## Name:
## Abstract

> .
>
> .


## Game Flow

> .
>
> .

## Graphics & Game Objects

### Sprites / Actors / Game Objects

> .
>
> .

### Background

> .
>
> .

## Scoring

## Game Screens


## Controls

